In [79]:
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import sys
import csv
import os
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

<ipython-input-79-bdeefdc2023d>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [ ]:
iterate = [1,2,3,4]
for count in iterate:
    print(f'''
    PRE-PROCESSING ON: StackSampleData/StackSample{count}.csv''' )
    stacksample1 = pd.read_csv(f'StackSampleData/StackSample{count}.csv',sep=";")
    stacksample1.head()
    print('Removing HTML tags')
    stacksample1["Body"] = stacksample1["Body"].progress_apply(lambda text: BeautifulSoup(text,'lxml').text)
    stacksample1.head()
    print('converting to lower case and removing useless stop words...')
    stacksample1["Body"] = stacksample1["Body"].str.lower()
    stacksample1["Title"] = stacksample1["Title"].str.lower()
    nltk.download("punkt")
    symbol_topics = ["c#", "c++", ".net", "asp.net", "node.js", "objective-c", "unity3d", "html5", "css3", \
                           "d3.js", "utf-8", "neo4j", "scikit-learn", "f#", "3d", "x86"]
    stacksample1["tokenized_body"] = stacksample1["Body"].progress_apply(lambda text: [word for word in nltk.word_tokenize(text)\
                                                                                      if word.isalpha() or word in list("+#")+\
                                                                                      symbol_topics])
    stacksample1["tokenized_title"] = stacksample1["Title"].progress_apply(lambda text: [word for word in nltk.word_tokenize(text)\
                                                                   if word.isalpha() or word in list("+#") + symbol_topics])

    stacksample1.head()
    mwe_tokenizer = nltk.MWETokenizer(separator="")
    mwe_tokenizer.add_mwe(("c", "#"))
    mwe_tokenizer.add_mwe(("c", "+", "+"))
    mwe_tokenizer.add_mwe(("f", "#"))
    stacksample1["tokenized_body"] = stacksample1["tokenized_body"].progress_apply(lambda tokens:\
                                                                                   [token for token in\
                                                                                    mwe_tokenizer.tokenize(tokens)])
    stacksample1["tokenized_title"] = stacksample1["tokenized_title"].progress_apply(lambda tokens: [token for token in\
                                                                                                    mwe_tokenizer.tokenize(tokens)])
    stacksample1.head()
    from nltk.corpus import stopwords
    nltk.download("stopwords")
    stop_words = set(stopwords.words("english"))

    def filter_stopwords(words):
        filtered_words = []
        for word in words:
            if word not in stop_words:
                filtered_words.append(word)
        return filtered_words

    stacksample1["tokenized_body"] = stacksample1["tokenized_body"].progress_apply(filter_stopwords)
    stacksample1["tokenized_title"] = stacksample1["tokenized_title"].progress_apply(filter_stopwords)
    pp.drop(["Unnamed: 0"],axis=1,inplace=True)
    stacksample1[["ID","tokenized_title","tokenized_body","Tags"]].to_csv(f"stacksample{count}pp.csv",sep=";")
    pp = pd.read_csv(f"stacksample{count}pp.csv",sep=";")
    pp.head()
    
    


    PRE-PROCESSING ON: StackSampleData/StackSample1.csv
Removing HTML tags


  0%|          | 0/250000 [00:00<?, ?it/s]